### Arquivo para teste e avaliação de clusters

In [2]:
#imports
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
sys.path.append('./clusters')

from clusters.clusters import *
from clusters.utils_clusters import *

# Testando os algoritmos de clustering sem redução de dimensionalidade

In [3]:
# parametros de teste
# dataset = 'Beauty'
# dataset = 'ML-1M'
# dataset = 'Steam'
# dataset = 'HomeKitchen'
dataset = 'PetSupplies'


# embedding_path = f'../data_preprocessing/{dataset}/{dataset}-similarity-values-thenlper_gte-large.pt'
embedding_path = f'../data_preprocessing/{dataset}/Pet-similarity-values-thenlper_gte-large.pt'
output_path = f'clusters/{dataset}/embeddings_completos'

os.makedirs(output_path, exist_ok=True)

embeddings = load_embeddings(embedding_path) 
print(f"Embeddings originais shape: {embeddings.shape}")

Embeddings originais shape: torch.Size([27447, 1000])


In [ ]:
#teste dos clusters com algoritmos que usam k

methods = [run_kmeans, run_agnes, run_fasterpam, run_fastermsc]
# methods = [run_agnes, run_fasterpam, run_fastermsc]

k_values = [6, 20, 50, 100, 200, 400, 750, 1000]


run_methods_with_k(
    embeddings=embeddings, 
    methods = methods, 
    k_values = k_values, 
    output_path = output_path, 
    dataset_name=dataset
)

Executando run_kmeans
 - k = 6
Clusters salvos em clusters/PetSupplies/embeddings_completos\run_kmeans\PetSupplies_cluster_run_kmeans_k6.csv
 - k = 10
Clusters salvos em clusters/PetSupplies/embeddings_completos\run_kmeans\PetSupplies_cluster_run_kmeans_k10.csv
 - k = 20
Clusters salvos em clusters/PetSupplies/embeddings_completos\run_kmeans\PetSupplies_cluster_run_kmeans_k20.csv
 - k = 30
Clusters salvos em clusters/PetSupplies/embeddings_completos\run_kmeans\PetSupplies_cluster_run_kmeans_k30.csv
 - k = 50
Clusters salvos em clusters/PetSupplies/embeddings_completos\run_kmeans\PetSupplies_cluster_run_kmeans_k50.csv
 - k = 70
Clusters salvos em clusters/PetSupplies/embeddings_completos\run_kmeans\PetSupplies_cluster_run_kmeans_k70.csv
 - k = 100
Clusters salvos em clusters/PetSupplies/embeddings_completos\run_kmeans\PetSupplies_cluster_run_kmeans_k100.csv
 - k = 150
Clusters salvos em clusters/PetSupplies/embeddings_completos\run_kmeans\PetSupplies_cluster_run_kmeans_k150.csv
 - k = 2

In [ ]:
# teste spectral clustering
k_values = [10, 50, 100, 300,500]
n_values = [20, 50, 75, 100, 150]                              

run_spectral_combinations(
    embeddings=embeddings,
    k_values=k_values,
    neighbor_values=n_values,
    output_path=output_path,
    dataset_name=dataset
)
#ultima execução - 756 minutos e 40 segundos

Executando run_spectral...
Executando SPECTRAL (k_values = 6, neighbor_values = 5)...


C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


Clusters salvos em clusters/PetSupplies/embeddings_completos\run_spectral/PetSupplies_clusters_run_spectral_k6_n5.csv
Executando SPECTRAL (k_values = 6, neighbor_values = 10)...


C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


Clusters salvos em clusters/PetSupplies/embeddings_completos\run_spectral/PetSupplies_clusters_run_spectral_k6_n10.csv
Executando SPECTRAL (k_values = 6, neighbor_values = 20)...


...

C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


Clusters salvos em clusters/PetSupplies/embeddings_completos\run_spectral/PetSupplies_clusters_run_spectral_k1000_n10.csv
Executando SPECTRAL (k_values = 1000, neighbor_values = 20)...


C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


Clusters salvos em clusters/PetSupplies/embeddings_completos\run_spectral/PetSupplies_clusters_run_spectral_k1000_n20.csv
Executando SPECTRAL (k_values = 1000, neighbor_values = 30)...
Clusters salvos em clusters/PetSupplies/embeddings_completos\run_spectral/PetSupplies_clusters_run_spectral_k1000_n30.csv
Executando SPECTRAL (k_values = 1000, neighbor_values = 50)...
Clusters salvos em clusters/PetSupplies/embeddings_completos\run_spectral/PetSupplies_clusters_run_spectral_k1000_n50.csv
Executando SPECTRAL (k_values = 1000, neighbor_values = 75)...
Clusters salvos em clusters/PetSupplies/embeddings_completos\run_spectral/PetSupplies_clusters_run_spectral_k1000_n75.csv
Executando SPECTRAL (k_values = 1000, neighbor_values = 100)...
Clusters salvos em clusters/PetSupplies/embeddings_completos\run_spectral/PetSupplies_clusters_run_spectral_k1000_n100.csv
Executando SPECTRAL (k_values = 1000, neighbor_values = 150)...
Clusters salvos em clusters/PetSupplies/embeddings_completos\run_spectra

: 

In [4]:
#teste dbscan
eps_values = [0.1, 0.2, 0.4, 0.8, 1.0]          
min_samples_values = [10, 20, 30, 50, 100, 200]

run_dbscan_combinations(
    embeddings=embeddings,
    eps_values=eps_values,
    min_samples_values=min_samples_values,
    output_path=output_path,
    dataset_name=dataset
)

Executando run_dbscan...
DBSCAN - eps = 0.1, min_samples = 10
Clusters salvos em clusters/PetSupplies/embeddings_completos\run_dbscan/PetSupplies_clusters_run_dbscan_eps0.1_min10.csv
DBSCAN - eps = 0.1, min_samples = 20
Clusters salvos em clusters/PetSupplies/embeddings_completos\run_dbscan/PetSupplies_clusters_run_dbscan_eps0.1_min20.csv
DBSCAN - eps = 0.1, min_samples = 30
Clusters salvos em clusters/PetSupplies/embeddings_completos\run_dbscan/PetSupplies_clusters_run_dbscan_eps0.1_min30.csv
DBSCAN - eps = 0.1, min_samples = 50
DBSCAN (eps=0.1, min_samples=50) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 100
DBSCAN (eps=0.1, min_samples=100) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 200
DBSCAN (eps=0.1, min_samples=200) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.2, min_samples = 10
DBSCAN (eps=0.2, min_samples=10) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.2, min_samples = 20
DBSCAN (eps=0.2, min_sam

In [5]:
#teste hdbscan
min_clusters_sizes = [20, 30, 100, 200, 300]
min_samples_values = [10, 20, 30, 50, 100]  
run_hdbscan_combinations(
    embeddings=embeddings,
    min_cluster_sizes=min_clusters_sizes,
    min_samples = min_samples_values,
    output_path=output_path,
    dataset_name=dataset
)

Executando run_hdbscan...
HDBSCAN - min_cluster_size = 20, min_samples = 10


C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Clusters salvos em clusters/PetSupplies/embeddings_completos\run_hdbscan/PetSupplies_clusters_run_hdbscan_minsize20_minsample10.csv
HDBSCAN - min_cluster_size = 20, min_samples = 20


C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Clusters salvos em clusters/PetSupplies/embeddings_completos\run_hdbscan/PetSupplies_clusters_run_hdbscan_minsize20_minsample20.csv
HDBSCAN - min_cluster_size = 20, min_samples = 30


...

HDBSCAN (min_cluster_size=300) gerou apenas um cluster ou ruído. Ignorado.
HDBSCAN - min_cluster_size = 300, min_samples = 50


C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


HDBSCAN (min_cluster_size=300) gerou apenas um cluster ou ruído. Ignorado.
HDBSCAN - min_cluster_size = 300, min_samples = 100


C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


HDBSCAN (min_cluster_size=300) gerou apenas um cluster ou ruído. Ignorado.
Clusterização concluída e resultados salvos


# Agora, vamos testar os algoritmos de clusterização com redução de dimensionalidade

In [6]:
# parametros de teste
# dataset = 'Beauty'
# dataset = 'ML-1M'
# dataset = 'Steam'
# dataset = 'HomeKitchen'
dataset = 'PetSupplies'


# embedding_path = f'../data_preprocessing/{dataset}/{dataset}-similarity-values-thenlper_gte-large.pt'
embedding_path = f'../data_preprocessing/{dataset}/Pet-similarity-values-thenlper_gte-large.pt'
embeddings = load_embeddings(embedding_path)
print(f"Embeddings originais shape: {embeddings.shape}")

methods_reducing = ['pca','umap']
n_components = [8,16,32,64,128,256,512]
# n_components = [4,8,16,32]


Embeddings originais shape: torch.Size([27447, 1000])


In [ ]:
# testando metodos com k usando redução de dimensionalidade
methods_clustering = [run_kmeans, run_agnes, run_fasterpam, run_fastermsc]

k_values = [5, 10, 30, 50, 100, 150, 200, 400, 500, 750, 1000]

for method_red in methods_reducing:
    for n in n_components:
        print(f'\n Embeddings reduzidos com {method_red} e n_components {n}')
        reduced_embeddings = dimensionality_reduction(embeddings=embeddings, method=method_red, n_components=n)
        if not isinstance(reduced_embeddings, torch.Tensor):
            reduced_embeddings = torch.tensor(reduced_embeddings, dtype=torch.float32)
        output_path = f'clusters/{dataset}/{method_red}/n_components{n}'
        os.makedirs(output_path, exist_ok=True)
        
        print('Metodos com numero de clusters (k) como parametro ...')

        run_methods_with_k(
            embeddings = reduced_embeddings, 
            methods = methods_clustering, 
            k_values = k_values, 
            output_path = output_path, 
            dataset_name=dataset
        )  
                        
print("\nTodos os testes para métodos com k fixo concluídos")


 Embeddings reduzidos com pca e n_components 8
Aplicando PCA com n_components = 8...
Dimensionalidade reduzida de 1000 para 8
Metodos com numero de clusters (k) como parametro ...
Executando run_kmeans
 - k = 5
Clusters salvos em clusters/PetSupplies/pca/n_components8\run_kmeans\PetSupplies_cluster_run_kmeans_k5.csv
 - k = 10
Clusters salvos em clusters/PetSupplies/pca/n_components8\run_kmeans\PetSupplies_cluster_run_kmeans_k10.csv
 - k = 30
Clusters salvos em clusters/PetSupplies/pca/n_components8\run_kmeans\PetSupplies_cluster_run_kmeans_k30.csv
 - k = 50
Clusters salvos em clusters/PetSupplies/pca/n_components8\run_kmeans\PetSupplies_cluster_run_kmeans_k50.csv
 - k = 100
Clusters salvos em clusters/PetSupplies/pca/n_components8\run_kmeans\PetSupplies_cluster_run_kmeans_k100.csv
 - k = 150
Clusters salvos em clusters/PetSupplies/pca/n_components8\run_kmeans\PetSupplies_cluster_run_kmeans_k150.csv
 - k = 200
Clusters salvos em clusters/PetSupplies/pca/n_components8\run_kmeans\PetSupp

In [ ]:
k_values = [10, 20, 50, 100, 300, 500]
n_values = [20, 30, 75, 100, 150]   

print('Testes para Spectral Clustering')
for method_red in methods_reducing:
    for n in n_components:
        print(f'\n Embeddings reduzidos com {method_red} e n_components {n}')
        reduced_embeddings = dimensionality_reduction(embeddings=embeddings, method=method_red, n_components=n)
        if not isinstance(reduced_embeddings, torch.Tensor):
            reduced_embeddings = torch.tensor(reduced_embeddings, dtype=torch.float32)
        output_path = f'clusters/{dataset}/{method_red}/n_components{n}'
        os.makedirs(output_path, exist_ok=True)               

        run_spectral_combinations(
            embeddings=reduced_embeddings,
            k_values=k_values,
            neighbor_values=n_values,
            output_path=output_path,
            dataset_name=dataset
        )

print("\nTodos os testes para Spectral Clustering concluídos")

Testes para Spectral Clustering

 Embeddings reduzidos com pca e n_components 4
Aplicando PCA com n_components = 4...
Dimensionalidade reduzida de 1000 para 4
Executando run_spectral...
Executando SPECTRAL (k_values = 10, neighbor_values = 20)...
Clusters salvos em clusters/ML-1M/pca/n_components4\run_spectral/ML-1M_clusters_run_spectral_k10_n20.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 30)...
Clusters salvos em clusters/ML-1M/pca/n_components4\run_spectral/ML-1M_clusters_run_spectral_k10_n30.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 50)...
Clusters salvos em clusters/ML-1M/pca/n_components4\run_spectral/ML-1M_clusters_run_spectral_k10_n50.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 75)...
Clusters salvos em clusters/ML-1M/pca/n_components4\run_spectral/ML-1M_clusters_run_spectral_k10_n75.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 100)...
Clusters salvos em clusters/ML-1M/pca/n_components4\run_spectral/ML-1M_clusters_run_spec

C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Dimensionalidade reduzida de 1000 para 4
Executando run_spectral...
Executando SPECTRAL (k_values = 10, neighbor_values = 20)...
Clusters salvos em clusters/ML-1M/umap/n_components4\run_spectral/ML-1M_clusters_run_spectral_k10_n20.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 30)...
Clusters salvos em clusters/ML-1M/umap/n_components4\run_spectral/ML-1M_clusters_run_spectral_k10_n30.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 50)...
Clusters salvos em clusters/ML-1M/umap/n_components4\run_spectral/ML-1M_clusters_run_spectral_k10_n50.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 75)...
Clusters salvos em clusters/ML-1M/umap/n_components4\run_spectral/ML-1M_clusters_run_spectral_k10_n75.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 100)...
Clusters salvos em clusters/ML-1M/umap/n_components4\run_spectral/ML-1M_clusters_run_spectral_k10_n100.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 150)...
Clusters salvos em clusters/ML-1M

C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Dimensionalidade reduzida de 1000 para 8
Executando run_spectral...
Executando SPECTRAL (k_values = 10, neighbor_values = 20)...
Clusters salvos em clusters/ML-1M/umap/n_components8\run_spectral/ML-1M_clusters_run_spectral_k10_n20.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 30)...
Clusters salvos em clusters/ML-1M/umap/n_components8\run_spectral/ML-1M_clusters_run_spectral_k10_n30.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 50)...
Clusters salvos em clusters/ML-1M/umap/n_components8\run_spectral/ML-1M_clusters_run_spectral_k10_n50.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 75)...
Clusters salvos em clusters/ML-1M/umap/n_components8\run_spectral/ML-1M_clusters_run_spectral_k10_n75.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 100)...
Clusters salvos em clusters/ML-1M/umap/n_components8\run_spectral/ML-1M_clusters_run_spectral_k10_n100.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 150)...
Clusters salvos em clusters/ML-1M

...

Dimensionalidade reduzida de 1000 para 128
Executando run_spectral...
Executando SPECTRAL (k_values = 10, neighbor_values = 20)...
Clusters salvos em clusters/ML-1M/umap/n_components128\run_spectral/ML-1M_clusters_run_spectral_k10_n20.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 30)...
Clusters salvos em clusters/ML-1M/umap/n_components128\run_spectral/ML-1M_clusters_run_spectral_k10_n30.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 50)...
Clusters salvos em clusters/ML-1M/umap/n_components128\run_spectral/ML-1M_clusters_run_spectral_k10_n50.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 75)...
Clusters salvos em clusters/ML-1M/umap/n_components128\run_spectral/ML-1M_clusters_run_spectral_k10_n75.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 100)...
Clusters salvos em clusters/ML-1M/umap/n_components128\run_spectral/ML-1M_clusters_run_spectral_k10_n100.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 150)...
Clusters salvos em cl

C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Dimensionalidade reduzida de 1000 para 256
Executando run_spectral...
Executando SPECTRAL (k_values = 10, neighbor_values = 20)...
Clusters salvos em clusters/ML-1M/umap/n_components256\run_spectral/ML-1M_clusters_run_spectral_k10_n20.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 30)...
Clusters salvos em clusters/ML-1M/umap/n_components256\run_spectral/ML-1M_clusters_run_spectral_k10_n30.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 50)...
Clusters salvos em clusters/ML-1M/umap/n_components256\run_spectral/ML-1M_clusters_run_spectral_k10_n50.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 75)...
Clusters salvos em clusters/ML-1M/umap/n_components256\run_spectral/ML-1M_clusters_run_spectral_k10_n75.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 100)...
Clusters salvos em clusters/ML-1M/umap/n_components256\run_spectral/ML-1M_clusters_run_spectral_k10_n100.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 150)...
Clusters salvos em cl

C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Dimensionalidade reduzida de 1000 para 512
Executando run_spectral...
Executando SPECTRAL (k_values = 10, neighbor_values = 20)...
Clusters salvos em clusters/ML-1M/umap/n_components512\run_spectral/ML-1M_clusters_run_spectral_k10_n20.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 30)...
Clusters salvos em clusters/ML-1M/umap/n_components512\run_spectral/ML-1M_clusters_run_spectral_k10_n30.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 50)...
Clusters salvos em clusters/ML-1M/umap/n_components512\run_spectral/ML-1M_clusters_run_spectral_k10_n50.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 75)...
Clusters salvos em clusters/ML-1M/umap/n_components512\run_spectral/ML-1M_clusters_run_spectral_k10_n75.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 100)...
Clusters salvos em clusters/ML-1M/umap/n_components512\run_spectral/ML-1M_clusters_run_spectral_k10_n100.csv
Executando SPECTRAL (k_values = 10, neighbor_values = 150)...
Clusters salvos em cl

In [ ]:
#teste dbscan
eps_values = [0.05, 0.1, 0.2, 0.4, 0.8, 1.0]          
min_samples_values = [10, 20, 30, 50, 100, 200]


print('Testes para DBSCAN Clustering')
for method_red in methods_reducing:
    for n in n_components:
        print(f'\n Embeddings reduzidos com {method_red} e n_components {n}')
        reduced_embeddings = dimensionality_reduction(embeddings=embeddings, method=method_red, n_components=n)
        if not isinstance(reduced_embeddings, torch.Tensor):
            reduced_embeddings = torch.tensor(reduced_embeddings, dtype=torch.float32)
        output_path = f'clusters/{dataset}/{method_red}/n_components{n}'
        os.makedirs(output_path, exist_ok=True)               

        run_dbscan_combinations(
            embeddings=reduced_embeddings,
            eps_values=eps_values,
            min_samples_values=min_samples_values,
            output_path=output_path,
            dataset_name=dataset
        )

print("\nTodos os testes para DBSCAN concluídos")

Testes para DBSCAN Clustering

 Embeddings reduzidos com pca e n_components 4
Aplicando PCA com n_components = 4...
Dimensionalidade reduzida de 1000 para 4
Executando run_dbscan...
DBSCAN - eps = 0.1, min_samples = 10
DBSCAN (eps=0.1, min_samples=10) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 20
DBSCAN (eps=0.1, min_samples=20) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 30
DBSCAN (eps=0.1, min_samples=30) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 50
DBSCAN (eps=0.1, min_samples=50) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 100
DBSCAN (eps=0.1, min_samples=100) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 200
DBSCAN (eps=0.1, min_samples=200) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.2, min_samples = 10
DBSCAN (eps=0.2, min_samples=10) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.2, min_samples = 20
DBSCAN (eps=0.

C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Dimensionalidade reduzida de 1000 para 4
Executando run_dbscan...
DBSCAN - eps = 0.1, min_samples = 10
Clusters salvos em clusters/ML-1M/umap/n_components4\run_dbscan/ML-1M_clusters_run_dbscan_eps0.1_min10.csv
DBSCAN - eps = 0.1, min_samples = 20
DBSCAN (eps=0.1, min_samples=20) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 30
DBSCAN (eps=0.1, min_samples=30) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 50
DBSCAN (eps=0.1, min_samples=50) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 100
DBSCAN (eps=0.1, min_samples=100) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 200
DBSCAN (eps=0.1, min_samples=200) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.2, min_samples = 10
Clusters salvos em clusters/ML-1M/umap/n_components4\run_dbscan/ML-1M_clusters_run_dbscan_eps0.2_min10.csv
DBSCAN - eps = 0.2, min_samples = 20
Clusters salvos em clusters/ML-1M/umap/n_components4\run_dbs

C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Dimensionalidade reduzida de 1000 para 8
Executando run_dbscan...
DBSCAN - eps = 0.1, min_samples = 10
Clusters salvos em clusters/ML-1M/umap/n_components8\run_dbscan/ML-1M_clusters_run_dbscan_eps0.1_min10.csv
DBSCAN - eps = 0.1, min_samples = 20
DBSCAN (eps=0.1, min_samples=20) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 30
DBSCAN (eps=0.1, min_samples=30) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 50
DBSCAN (eps=0.1, min_samples=50) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 100
DBSCAN (eps=0.1, min_samples=100) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 200
DBSCAN (eps=0.1, min_samples=200) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.2, min_samples = 10
Clusters salvos em clusters/ML-1M/umap/n_components8\run_dbscan/ML-1M_clusters_run_dbscan_eps0.2_min10.csv
DBSCAN - eps = 0.2, min_samples = 20
Clusters salvos em clusters/ML-1M/umap/n_components8\run_dbs

...

Dimensionalidade reduzida de 1000 para 128
Executando run_dbscan...
DBSCAN - eps = 0.1, min_samples = 10
Clusters salvos em clusters/ML-1M/umap/n_components128\run_dbscan/ML-1M_clusters_run_dbscan_eps0.1_min10.csv
DBSCAN - eps = 0.1, min_samples = 20
DBSCAN (eps=0.1, min_samples=20) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 30
DBSCAN (eps=0.1, min_samples=30) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 50
DBSCAN (eps=0.1, min_samples=50) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 100
DBSCAN (eps=0.1, min_samples=100) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 200
DBSCAN (eps=0.1, min_samples=200) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.2, min_samples = 10
Clusters salvos em clusters/ML-1M/umap/n_components128\run_dbscan/ML-1M_clusters_run_dbscan_eps0.2_min10.csv
DBSCAN - eps = 0.2, min_samples = 20
Clusters salvos em clusters/ML-1M/umap/n_components128

C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Dimensionalidade reduzida de 1000 para 256
Executando run_dbscan...
DBSCAN - eps = 0.1, min_samples = 10
Clusters salvos em clusters/ML-1M/umap/n_components256\run_dbscan/ML-1M_clusters_run_dbscan_eps0.1_min10.csv
DBSCAN - eps = 0.1, min_samples = 20
DBSCAN (eps=0.1, min_samples=20) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 30
DBSCAN (eps=0.1, min_samples=30) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 50
DBSCAN (eps=0.1, min_samples=50) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 100
DBSCAN (eps=0.1, min_samples=100) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 200
DBSCAN (eps=0.1, min_samples=200) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.2, min_samples = 10
Clusters salvos em clusters/ML-1M/umap/n_components256\run_dbscan/ML-1M_clusters_run_dbscan_eps0.2_min10.csv
DBSCAN - eps = 0.2, min_samples = 20
Clusters salvos em clusters/ML-1M/umap/n_components256

C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Dimensionalidade reduzida de 1000 para 512
Executando run_dbscan...
DBSCAN - eps = 0.1, min_samples = 10
Clusters salvos em clusters/ML-1M/umap/n_components512\run_dbscan/ML-1M_clusters_run_dbscan_eps0.1_min10.csv
DBSCAN - eps = 0.1, min_samples = 20
DBSCAN (eps=0.1, min_samples=20) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 30
DBSCAN (eps=0.1, min_samples=30) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 50
DBSCAN (eps=0.1, min_samples=50) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 100
DBSCAN (eps=0.1, min_samples=100) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.1, min_samples = 200
DBSCAN (eps=0.1, min_samples=200) gerou 1 cluster ou só ruído, ignorado.
DBSCAN - eps = 0.2, min_samples = 10
Clusters salvos em clusters/ML-1M/umap/n_components512\run_dbscan/ML-1M_clusters_run_dbscan_eps0.2_min10.csv
DBSCAN - eps = 0.2, min_samples = 20
Clusters salvos em clusters/ML-1M/umap/n_components512

In [ ]:
#teste hdbscan
min_clusters_sizes = [30, 50, 100, 200, 300]
min_samples_values = [10, 20, 30, 50, 100]   
 

print('Testes para HDBSCAN Clustering')
for method_red in methods_reducing:
    for n in n_components:
        print(f'\n Embeddings reduzidos com {method_red} e n_components {n}')
        reduced_embeddings = dimensionality_reduction(embeddings=embeddings, method=method_red, n_components=n)
        if not isinstance(reduced_embeddings, torch.Tensor):
            reduced_embeddings = torch.tensor(reduced_embeddings, dtype=torch.float32)
        output_path = f'clusters/{dataset}/{method_red}/n_components{n}'
        os.makedirs(output_path, exist_ok=True)               

        run_hdbscan_combinations(
            embeddings=reduced_embeddings,
            min_cluster_sizes=min_clusters_sizes,
            min_samples = min_samples_values,
            output_path=output_path,
            dataset_name=dataset
        )

print("\nTodos os testes para HDBSCAN concluídos")

Testes para HDBSCAN Clustering

 Embeddings reduzidos com pca e n_components 4
Aplicando PCA com n_components = 4...
Dimensionalidade reduzida de 1000 para 4
Executando run_hdbscan...
HDBSCAN - min_cluster_size = 20, min_samples = 10
Clusters salvos em clusters/ML-1M/pca/n_components4\run_hdbscan/ML-1M_clusters_run_hdbscan_minsize20_minsample10.csv


C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


HDBSCAN - min_cluster_size = 20, min_samples = 20
Clusters salvos em clusters/ML-1M/pca/n_components4\run_hdbscan/ML-1M_clusters_run_hdbscan_minsize20_minsample20.csv


C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


HDBSCAN - min_cluster_size = 20, min_samples = 30
HDBSCAN (min_cluster_size=20) gerou apenas um cluster ou ruído. Ignorado.
HDBSCAN - min_cluster_size = 20, min_samples = 50
Clusters salvos em clusters/ML-1M/pca/n_components4\run_hdbscan/ML-1M_clusters_run_hdbscan_minsize20_minsample50.csv


...

Clusters salvos em clusters/ML-1M/umap/n_components512\run_hdbscan/ML-1M_clusters_run_hdbscan_minsize300_minsample50.csv
HDBSCAN - min_cluster_size = 300, min_samples = 100


C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


HDBSCAN (min_cluster_size=300) gerou apenas um cluster ou ruído. Ignorado.
HDBSCAN - min_cluster_size = 300, min_samples = 200


C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\cruzd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


HDBSCAN (min_cluster_size=300) gerou apenas um cluster ou ruído. Ignorado.
Clusterização concluída e resultados salvos

Todos os testes para HDBSCAN concluídos
